In [2]:
import pandas as pd
from datasets import load_dataset

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


# Betöltés

betöltjük a klasszifikációs adathalmazt

In [3]:
dataset = load_dataset('boapps/kmdb_classification')

# Adathalmaz felfedezése

Látható, hogy három split-ből áll az adathalmaz

In [ ]:
{split: len(dataset[split]) for split in dataset}

A label adja meg, hogy adott cikk korrupciós témájú

In [5]:
dataset.unique('label')

{'test': [0, 1], 'train': [0, 1], 'validation': [1, 0]}

A validációs halmaz csak kézzel annotált mintákat tartalmaz

In [4]:
dataset.unique('is_hand_annoted')

{'test': [True], 'train': [False, True], 'validation': [True, False]}

In [6]:
dataset['train'].filter(lambda row: not row['is_hand_annoted'])

Filter:   0%|          | 0/43060 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections'],
    num_rows: 13795
})

## Oszlopok magyarázata:

`text`: cikk törzse

`title`: hírportál által adott cím

`description`: hírportál által adott lead (kis bevezető/összefoglaló az elején)

`keywords`: hírportál címkék (nem mindig van és nem mindig értelmes)

`label`: korrupcióról szól-e a cikk (típus: int, 1 ha igen, 0 egyébként)

`url`: cikk url-je

`authors`: cikk szerzői, címkékhez hasonlóan, nem minden esetben van meg

`date`: cikk megjelenésének ideje, változó pontossággal és ritkán értelmetlen

`kmonitor_description`: csak korrupciós cikkek esetében, k-monitor adatbázisában tárolt leírás (sokszor egyezik a rendes leírással)

`kmonitor_title`: ugyanez címmel

`kmonitor_tags`: ez a keywords-el ellentétben K-Monitoros önkéntesek általi címkézés, meghatározott "címke halmazból" (ebben vannak még helyszínek, amiket people-höz és institutes-hoz hasonlóan szét lehetne (kéne) szedni egy places oszlopba)

`institutes`: cikkben megjelenő intézmények, K-Monitoros gyűjtés

`people`: ugyanez személyekkel

`is_hand_annoted`: azt jelöli, hogy adott cikket ember, vagy egy nyelvi modell annotálta, pozitív minták esetében mindig True

In [ ]:
dataset['train'].features

Keressünk helyreigazításokat!

In [ ]:
dataset['train'].filter(lambda article: 'helyreigazítás' in article['title'].lower() or 'helyreigazítás' in article['keywords'])['title']

# Pandas

Az adathalmazt átalakítjuk pandas objektumokká a további feldolgozás egyszerűsítése miatt

In [ ]:
dfs = {split: dataset[split].to_pandas() for split in dataset.keys()}

In [ ]:
dfs['test']

train: 62% pozitív (korrupciós) és 38% negatív minta

validation: 51.68% pozitív (korrupciós) és 48.31% negatív

test: 50-50%

In [ ]:
for split in dataset.keys():
    print(split, dfs[split].groupby(['label']).size(), sep='\n', end='\n\n')

In [ ]:
full_df = pd.concat(dfs.values())

In [ ]:
full_df['kmonitor_tags'].apply(pd.Series).unstack().reset_index(drop=True).dropna().value_counts()

In [ ]:
full_df['authors'].apply(pd.Series).unstack().reset_index(drop=True).dropna().value_counts()

~7500 cikk megjelenési ideje hiányzik

In [ ]:
full_df['date'].isna().sum()